# Reconnaissance faciale via $k$ plus proches voisins ($k$-NN)

>## Partie 2: Reconnaissance faciale en temps réel

## Librairies

In [1]:
import pickle
import numpy as np

import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

## Algorithme

In [2]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

In [3]:
N = len(noms)

visages = np.array(visages).reshape(N, 50, 50, 3)

X_train, X_test, y_train, y_test = train_test_split(visages, noms, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(np.unique(y_train_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=10, validation_data=(X_test, y_test_encoded))

test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print(f'Accuracy du modèle sur l\'ensemble de test : {test_acc:.2f}')


Epoch 1/10
1/1 [==============================] - 0s 248ms/step - loss: 87.2421 - accuracy: 0.4688 - val_loss: 616.3860 - val_accuracy: 0.3750
Epoch 2/10
1/1 [==============================] - 0s 26ms/step - loss: 427.5522 - accuracy: 0.5312 - val_loss: 429.9034 - val_accuracy: 0.3750
Epoch 3/10
1/1 [==============================] - 0s 25ms/step - loss: 299.0457 - accuracy: 0.5312 - val_loss: 182.3597 - val_accuracy: 0.3750
Epoch 4/10
1/1 [==============================] - 0s 27ms/step - loss: 127.6629 - accuracy: 0.5312 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 24ms/step - loss: 3.8405e-06 - accuracy: 1.0000 - val_loss: 43.3186 - val_accuracy: 0.6250
Epoch 6/10
1/1 [==============================] - 0s 23ms/step - loss: 64.2053 - accuracy: 0.4688 - val_loss: 41.9304 - val_accuracy: 0.6250
Epoch 7/10
1/1 [==============================] - 0s 24ms/step - loss: 60.6159 - accuracy: 0.4688 - val_loss: 25.5276 - val_accuracy: 0.6250


## Exécution

In [4]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)

while True:
    ret, trame = camera.read()

    if ret:
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            visage = cv2.resize(trame[y:y + h, x:x + l], (50, 50))
            visage = np.expand_dims(visage, axis=0)  # Ajout d'une dimension pour correspondre à la forme attendue du modèle
            visage = visage / 255.0  # Normalisation des pixels

            # Prédiction avec le modèle CNN
            prediction = model.predict(visage)
            classe = np.argmax(prediction)
            probabilite = prediction[0][classe]

            # Affichage du résultat
            cv2.putText(trame, f"{label_encoder.classes_[classe]} ({probabilite:.2f})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + h), (0, 0, 255), 2)

        cv2.imshow('Reconnaissance faciale en temps réel avec CNN', trame)

        if cv2.waitKey(1) == 27:
            break
    else:
        print("Erreur")
        break

cv2.destroyAllWindows()
camera.release()

2024-03-01 13:40:41.240 Python[41239:4202574] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


1/1 [==============================] - 0s 7ms/step
